In [35]:
# Tell python where to look for modules. 
# Depending on how your jupyter handles working directories, this may not be needed.
import sys
sys.path.append('../../hourly-egrid/')

# import the necessary packages
%reload_ext autoreload
%autoreload 2

import os
import pandas as pd
import plotly.express as px
import numpy as np

import src.data_cleaning as dc
import src.load_data as ld

In [36]:
df_923_emissions, primary_fuel_table = dc.clean_eia923(2020, include_nox=True, include_so2=True, include_co2e=True)

Removing 0 plants that are not grid-connected
Removing 0 plants located in the following states: ['PR']


In [37]:
df_923_emissions

,report_date,plant_id_eia,generator_id,net_generation_mwh,fuel_consumed_mmbtu,fuel_consumed_for_electricity_mmbtu,co2_mass_tons,co2_mass_tons_adjusted,nox_mass_lbs,nox_mass_lbs_adjusted,so2_mass_lbs,so2_mass_lbs_adjusted,co2e_mass_tons,co2e_mass_tons_adjusted
0,2020-01-01,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2020-01-01,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2020-01-01,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2020-01-01,1,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2020-01-01,1,WT1,15.1585,133.0,133.0,0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284171,2020-12-01,64816,GEN1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284172,2020-12-01,64816,GEN2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284173,2020-12-01,64816,GEN3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
284174,2020-12-01,64836,CATAL,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [39]:
no_net_generation = df_923_emissions[pd.isna(df_923_emissions['net_generation_mwh'])]

In [41]:
df_923_plant_emissions = df_923_emissions.groupby(['plant_id_eia', 'report_date']).sum()

In [42]:
df_923_plant_emissions

net_generation_mwh  fuel_consumed_mmbtu  fuel_consumed_for_electricity_mmbtu  co2_mass_tons  co2_mass_tons_adjusted  nox_mass_lbs  nox_mass_lbs_adjusted  so2_mass_lbs  so2_mass_lbs_adjusted  co2e_mass_tons  co2e_mass_tons_adjusted
plant_id_eia report_date                                                                                                                                                                                                                                        
1            2020-01-01               30.317                266.0                                266.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-02-01               31.076                273.0                                273.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-03-01               31.133                273.0                                273.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-04-01               30.983                272.0                                272.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-05-01               29.431                258.0                                258.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
...                                      ...                  ...                                  ...            ...                     ...           ...                    ...           ...                    ...             ...                      ...
64837        2020-08-01                0.000                  0.0                                  0.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-09-01                0.000                  0.0                                  0.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-10-01                0.000                  0.0                                  0.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-11-01                0.000                  0.0                                  0.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0
             2020-12-01                0.000                  0.0                                  0.0            0.0                     0.0           0.0                    0.0           0.0                    0.0             0.0                      0.0

[126666 rows x 11 columns]

In [45]:
def export_cleaned_eia_923(year):
    """Helper function for exporting a year of cleaned EIA-923 data."""
    print(f'[INFO] Cleaning EIA-923 data for {year}')
    df_923_generator_emissions, primary_fuel_table = dc.clean_eia923(2020, include_nox=True, include_so2=True, include_co2e=True)
    print('[INFO] Done cleaning.')
    # Group by EIA plant ID and sum to the plant level.
    df_923_plant_emissions = df_923_emissions.groupby(['plant_id_eia', 'report_date']).sum()
    
    # Export the data to output.
    output_path = f'output/923/eia_923_generation_and_emissions_{year}.csv'
    df_923_plant_emissions.to_csv(output_path)
    print('[INFO] Wrote to:', output_path)
    
    output_path = f'output/923/eia_923_primary_fuel_table_{year}.csv'
    primary_fuel_table.to_csv(output_path)
    print('[INFO] Wrote to:', output_path)
    print('Done')

In [ ]:
# Export across all available years.
years_to_export = range(2001, 2021)
for year in years_to_export:
    export_cleaned_eia_923(year)

[INFO] Cleaning EIA-923 data for 2001
